In [73]:
import os
os.environ["MISTRAL_API_KEY"] = "NXyKdE5JFehmTjXn1RtYyVBOlMzPLGyB"
print(f"MISTRAL_API_KEY: {os.environ.get('MISTRAL_API_KEY')}")

MISTRAL_API_KEY: NXyKdE5JFehmTjXn1RtYyVBOlMzPLGyB


In [74]:
api_key = os.getenv("MISTRAL_API_KEY")

In [75]:
import requests
from bs4 import BeautifulSoup
import re

policyLinks = [
    "https://www.udst.edu.qa/about-udst/institutional-excellence-ie/policies-and-procedures/student-conduct-policy",
    "https://www.udst.edu.qa/about-udst/institutional-excellence-ie/udst-policies-and-procedures/academic-schedule-policy",
    "https://www.udst.edu.qa/about-udst/institutional-excellence-ie/policies-and-procedures/student-attendance-policy",
    "https://www.udst.edu.qa/about-udst/institutional-excellence-ie/policies-and-procedures/admissions-policy",
    "https://www.udst.edu.qa/about-udst/institutional-excellence-ie/policies-and-procedures/registration-policy",
    "https://www.udst.edu.qa/about-udst/institutional-excellence-ie/policies-and-procedures/sport-and-wellness-facilities-and",
    "https://www.udst.edu.qa/about-udst/institutional-excellence-ie/udst-policies-and-procedures/international-student-policy",
    "https://www.udst.edu.qa/about-udst/institutional-excellence-ie/udst-policies-and-procedures/student-counselling-services-policy",
    "https://www.udst.edu.qa/about-udst/institutional-excellence-ie/udst-policies-and-procedures/graduate-academic-standing-policy",
    "https://www.udst.edu.qa/about-udst/institutional-excellence-ie/policies-and-procedures/use-library-space-policy",
    "https://www.udst.edu.qa/about-udst/institutional-excellence-ie/policies-and-procedures/scholarship-and-financial-assistance"
]

all_texts = {}

for url in policyLinks:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    text = soup.get_text(separator="\n", strip=True)
    all_texts[url] = text
    print(f"Extracted text from: {url}")



Extracted text from: https://www.udst.edu.qa/about-udst/institutional-excellence-ie/policies-and-procedures/student-conduct-policy
Extracted text from: https://www.udst.edu.qa/about-udst/institutional-excellence-ie/udst-policies-and-procedures/academic-schedule-policy
Extracted text from: https://www.udst.edu.qa/about-udst/institutional-excellence-ie/policies-and-procedures/student-attendance-policy
Extracted text from: https://www.udst.edu.qa/about-udst/institutional-excellence-ie/policies-and-procedures/admissions-policy
Extracted text from: https://www.udst.edu.qa/about-udst/institutional-excellence-ie/policies-and-procedures/registration-policy
Extracted text from: https://www.udst.edu.qa/about-udst/institutional-excellence-ie/policies-and-procedures/sport-and-wellness-facilities-and
Extracted text from: https://www.udst.edu.qa/about-udst/institutional-excellence-ie/udst-policies-and-procedures/international-student-policy
Extracted text from: https://www.udst.edu.qa/about-udst/ins

In [ ]:
file_name = "Text.txt"
with open(file_name, 'w') as file:
    file.write(text)

In [77]:
chunk_size = 512
chunks = [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]

In [78]:
len(chunks)

40

In [79]:
from mistralai import Mistral
def get_text_embedding(list_txt_chunks):
    client = Mistral(api_key=api_key)
    embeddings_batch_response = client.embeddings.create(model="mistral-embed", inputs=list_txt_chunks)
    return embeddings_batch_response.data

In [80]:
import numpy as np
text_embeddings = get_text_embedding(chunks)

In [81]:
len(text_embeddings)

40

In [82]:
len(text_embeddings[0].embedding)

1024

In [83]:
embeddings = np.array([text_embeddings[i].embedding for i in range(len(text_embeddings))])

In [84]:
import faiss

d = len(text_embeddings[0].embedding)
index = faiss.IndexFlatL2(d)
index.add(embeddings)

In [85]:
question = "How to get a scholarship"
question_embeddings = np.array([get_text_embedding([question])[0].embedding])

In [86]:
question_embeddings

array([[-0.03778076,  0.03665161,  0.00070381, ..., -0.01670837,
        -0.00723267,  0.01064301]])

In [87]:
D, I = index.search(question_embeddings, k=10)
print(I)

[[36 22 18 27  7 24 23 34 32 35]]


In [88]:
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
print(retrieved_chunk)

[' and pay tuition fees to the University can receive their Scholarship Award while participating in the exchange program.\n9.4 Non-Tuition Expenses\nScholarship Awards provided by the University are applied to the University’s tuition and related fees only and cannot be applied to non-tuition related expenses including but not limited to transportation, meals, books, and personal supplies.\n9.5 Retroactive Awards\nScholarship Awards provided by the University cannot be applied retroactively.\n\nThe Award becomes a', 'ncial Assistance Award.\n5.0 Scholarship Eligibility\n5.1 Scholarship Application and Awards\n5.1.1 Applicants must meet or exceed the minimum criteria for Scholarship Award consideration per the Scholarship and Financial Assistance Procedure (PR-ST-11).\n5.1.2 As Scholarship categories vary, applications will be evaluated by the Scholarship Committee per the eligibility requirements outlined in the Scholarship and Financial Assistance Procedure (PR-ST-11).\n5.1.3 The Sch

In [89]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [90]:
from mistralai import Mistral, UserMessage
def mistral(user_message, model="mistral-small-latest", is_json=False):
    model = "mistral-large-latest"
    client = Mistral(api_key=api_key)
    messages = [
        UserMessage(content=user_message),
    ]
    chat_response = client.chat.complete(
        model=model,
        messages=messages,
        )
    return chat_response.choices[0].message.content

In [91]:
response = mistral(prompt)
print(response)

To get a scholarship at the University, you should follow these steps based on the provided context:

1. **Meet Minimum Criteria**: Ensure that you meet or exceed the minimum criteria for Scholarship Award consideration as outlined in the Scholarship and Financial Assistance Procedure (PR-ST-11).

2. **Apply for Scholarship**: Submit a scholarship application. The application will be evaluated by the Scholarship Committee based on the eligibility requirements outlined in the Scholarship and Financial Assistance Procedure (PR-ST-11).

3. **Scholarship Categories**:
   - **Academic Scholarship**: Demonstrate academic merit based on your academic success in high school and University grades.
   - **Talent Scholarship**: Show distinguished ability and achievements in either sport, arts, or the sciences, alongside academic performance.

4. **Adhere to Policies**: If awarded a scholarship, adhere to the University's policies and regulations, including the Student Conduct Policy (PL-ST-01), a